In [3]:
import pydicom
import numpy as np
import matplotlib.pyplot as plt

# Path to your DICOM file
dcm_file_path = "radioprotect/data/SAMPLE_001/RT.1.2.246.352.221.476609808337909324018268205780166052003.dcm"

# Load the DICOM file
try:
    dcm = pydicom.dcmread(dcm_file_path)
except Exception as e:
    print(f"Error loading DICOM file: {e}")
    exit()

In [5]:
# Extract pixel data as a NumPy array
if hasattr(dcm, "pixel_array"):
    image = dcm.pixel_array
    print(f"Image shape: {image.shape}")
else:
    print("No pixel data found in the DICOM file.")
    exit()

# Optional: Display the CT image
plt.imshow(image, cmap="gray")
plt.title("CT Scan")
plt.axis("off")  # Hide axes
plt.show()

# Optional: Access metadata (e.g., Patient ID, Slice Thickness)
print(f"Patient ID: {dcm.get('PatientID', 'N/A')}")
print(f"Slice Thickness: {dcm.get('SliceThickness', 'N/A')} mm")


No pixel data found in the DICOM file.


NameError: name 'image' is not defined

: 

In [6]:
dcm.keys
dcm.Modality

'RTRECORD'

In [6]:
from pathlib import Path

p = Path('radioprotect/data/SAMPLE_002')

contents = list(p.iterdir())

In [7]:
for c in contents:
    if "dcm" not in c.name:
        print(c.name)